In [42]:
import pandas as pd
import numpy as np
import mysql.connector

mydb1 = mysql.connector.connect(
    host="127.0.0.1",
    port="3306",
    user="root",
    password="2389",
    database="staging"
)
cur = mydb1.cursor()
# Lấy dữ liệu từ bảng employees
cur.execute("SELECT employee_id, last_name, first_name, job_title FROM employees")
results = cur.fetchall()
df_employees = pd.DataFrame(results, columns=[i[0] for i in cur.description])

# Lấy dữ liệu từ bảng customers
cur.execute("SELECT customer_id, city, country, sales_rep FROM customers")
results = cur.fetchall()
df_customers = pd.DataFrame(results, columns=[i[0] for i in cur.description])

# Lấy dữ liệu từ bảng payments
cur.execute("SELECT customer_id, amount FROM payments")
results = cur.fetchall()
df_payments = pd.DataFrame(results, columns=[i[0] for i in cur.description])

# Lấy dữ liệu từ bảng orders
cur.execute("SELECT order_id, customer_id, order_date FROM orders")
results = cur.fetchall()
df_orders = pd.DataFrame(results, columns=[i[0] for i in cur.description])

# Lấy dữ liệu từ bảng products
cur.execute("SELECT product_code, product_name, product_line FROM products")
results = cur.fetchall()
df_products = pd.DataFrame(results, columns=[i[0] for i in cur.description])

# Lấy dữ liệu từ bảng productlines
cur.execute("SELECT product_line FROM productlines")
results = cur.fetchall()
df_productlines = pd.DataFrame(results, columns=[i[0] for i in cur.description])

# Lấy dữ liệu từ bảng orderdetails
cur.execute("SELECT order_id, product_code, quantity_ordered, price_each FROM orderdetails")
results = cur.fetchall()
df_orderdetails = pd.DataFrame(results, columns=[i[0] for i in cur.description])

# Đóng kết nối
cur.close()
mydb1.close()

In [43]:

# lấy bảng dimention time
'''# Lấy dữ liệu từ bảng orders
cur.execute("SELECT order_id, customer_id, order_date FROM orders")
results = cur.fetchall()
df_orders = pd.DataFrame(results, columns=[i[0] for i in cur.description])
'''

dfm_time=df_orders[['order_date']].copy()

# tao bang dimention time 
dfm_time['order_date'] = pd.to_datetime(dfm_time['order_date'])

dfm_time['id_time'] = dfm_time['order_date'].dt.to_period('M')
dfm_time=dfm_time.drop(columns='order_date')
dfm_time=dfm_time.drop_duplicates()
dfm_time['year']=dfm_time['id_time'].dt.year
dfm_time['month']=dfm_time['id_time'].dt.month

#--------- tao xong dataframe dimension time -----------
# lấy  bảng dimension khu vực 
dfm_kv=df_customers[['city','country']].copy()
dfm_kv=dfm_kv.drop_duplicates()
# tao xong bang dataframe dimention khu vuc
# bang dimention sanpham df_products
#bang dimension nhanvien df_employees
dfm_time['id_time']=dfm_time['id_time'].dt.strftime('%Y-%m')

def init_kv(row):
    city = ""
    country = ""

    # Kiểm tra nếu 'city' và 'country' không phải NaN
    if pd.notna(row['city']):
        city = row['city']    
        if pd.notna(row['country']):
            country = row['country']

    # Trả về kết quả kết hợp city và country
    return city + "-" + country

dfm_kv['id_kv']=dfm_kv.apply(init_kv,axis=1)


i=2
# doi thu tu cot
cols=list(dfm_kv.columns)
cols=[cols[i]]+[col for col in cols if col!=cols[i]]
dfm_kv=dfm_kv[cols]

In [ ]:
# tao bang cac bang dimension

mydb1 = mysql.connector.connect(
    host="127.0.0.1",
    port="3306",
    user="root",
    password="2389",
    database="staging"
)
cur = mydb1.cursor()
try:
    cur.execute("""
                CREATE TABLE IF NOT EXISTS dim_kv (
                    id_kv varchar(100),
                    city varchar(20),
                    country varchar(20)
                )
                """)
    cur.execute("""
                 CREATE TABLE IF NOT EXISTS dim_time(
                     id_time varchar(100),
                     year int,
                     month int
                )
                """)
    # chen du lieu vao bang dim_kv,dimtime
    Values_kv=[tuple(x) for x in dfm_kv.values]
    Values_time=[tuple(x) for x in dfm_time.values]

    try:
        cur.executemany("INSERT INTO dim_kv VALUES(%s,%s,%s)",Values_kv)
        cur.executemany("INSERT INTO dim_time VALUES (%s,%s,%s)",Values_time)
        mydb1.commit()
        
    except Exception as e:
        print(e)

except:
    pass


In [22]:
# join order vaf customer

df_co=pd.merge(df_customers,df_orders,on='customer_id',how='inner')
df_co.head(5)

df_co=pd.merge(df_co,df_orderdetails,left_on='order_id',right_on='order_id',how='inner')

# tao id time trong bang fact
df_co['order_date']=pd.to_datetime(df_co['order_date'])
df_co['id_time']=df_co['order_date'].dt.to_period("M")
df_co=df_co.drop(columns='order_date')

# tao id_khu vuc o bang fact

def renuave(row):
    return float(row['price_each']*row['quantity_ordered'])

              
df_co['id_kv']=df_co.apply(init_kv,axis=1)
df_co['renuave']=df_co.apply(renuave,axis=1)
df_co=df_co.drop(columns=['city','country'])

df_fact=df_co.groupby(['sales_rep','id_kv','id_time']).agg({
                      'renuave':'sum',
                      'quantity_ordered':'sum'
}).reset_index()
df_fact['id_time'] = df_fact['id_time'].dt.strftime('%Y-%m')

In [ ]:


# tao bang fact_sale_order

def decimal(x):
    try:
        return float(x)
    except Exception as e:
        print(f"Error converting {x} to float: {e}")
        return 0.0  


df_fact['renuave'] = df_fact['renuave'].apply(decimal)

# tao ID bang fact so nguyen tang dan
cnt=1000
def set_id(x):
    global cnt
    cnt=cnt+1
    return cnt

df_fact['id']=df_fact.apply(set_id,axis=1)
# di chuyen cot 5 torng bang fact len dau
i = 5  # Chỉ số cột bạn muốn di chuyển
cols = list(df_fact.columns)

cols = [cols[i]] + [col for col in cols if col != cols[i]]

df_fact = df_fact[cols]

print(df_fact.head())

mydb1 = mysql.connector.connect(
    host="127.0.0.1",
    port="3306",
    user="root",
    password="2389",
    database="staging"
)
cur = mydb1.cursor()

Values = [tuple(x) for x in df_fact.values]

try:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS FACT1(
            ID INT,
            sales_rep INT,
            id_kv varchar(100),
            id_time varchar(10),
            renuave float,
            quantity_ordered INT
        )
    """)
except Exception as e:
    print("Error creating table:", e)

# Chèn dữ liệu vào bảng
try:
    cur.executemany("INSERT INTO FACT1 (ID,sales_rep, id_kv, id_time, renuave, quantity_ordered) VALUES (%s,%s,%s, %s, %s, %s)", Values)
    mydb1.commit()  
    print(f"{cur.rowcount} rows inserted successfully")
except Exception as e:
    print("Error inserting data:", e)





In [ ]:
# tao bang fact doanh so san pham
mydb1 = mysql.connector.connect(
    host="127.0.0.1",
    port="3306",
    user="root",
    password="2389",
    database="staging"
)

cur = mydb1.cursor()
# set id bang fact san pham
id=10000
def set_id_fact_sp(x):
    global id
    id=id+1
    return id

df_factsp=df_co.groupby(['product_code','id_kv']).agg({'quantity_ordered':'sum'}).reset_index()
df_factsp['id']=df_factsp.apply(set_id_fact_sp,axis=1)

i=3 # thay doi vi tri cot trong bang fact_sp
cols=list(df_factsp.columns)
cols=[cols[i]] + [col for col in cols if col!=cols[i]]
df_factsp=df_factsp[cols]

try:
    cur.execute("""
                    CREATE TABLE IF NOT EXISTS FACT_SP (
                             ID INT,
                             product_code VARCHAR(15),
                             id_kv VARCHAR(60),
                             quantity_ordered INT
                )       
                """)
except:
   pass
try:
    values=[tuple(x) for x in df_factsp.values]
    cur.executemany("INSERT INTO FACT_SP VALUES (%s,%s,%s,%s)",values)
    
    mydb1.commit()
    print("insert complete")
except Exception as e:
    print(e)
        

